In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import cv2
import imageio
import getpass

import azure.storage.blob
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


from tensorflow.keras.models import Model
import tensorflow.keras as keras
import tensorflow as tf
from keras.callbacks import Callback

import sys
sys.path.append('../ml-workflow/') # point to wherever the ml-workflow directory is
import imp

import data_utils as du
import ml_models

from tqdm.auto import trange, tqdm
import concurrent.futures

# Need to comment out gray_image = cv2.cvtColor(ri, cv2.COLOR_BGR2GRAY) in du.prepocess_input
# gray_image = ri
from importlib import reload
reload(du)


<module 'data_utils' from 'C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\../ml-workflow\\data_utils.py'>

## Current Azure API Key

In [4]:
"DefaultEndpointsProtocol=https;AccountName=ifcb;AccountKey=mzeoQjV8g8RZyI18AdD4x1XxQRm0GesvssNtPgGNom7a8Kk3FRe8bjCm9BRq7m/gwqrK8GcwOXDBPKWq21jFPg==;EndpointSuffix=core.windows.net"

'DefaultEndpointsProtocol=https;AccountName=ifcb;AccountKey=mzeoQjV8g8RZyI18AdD4x1XxQRm0GesvssNtPgGNom7a8Kk3FRe8bjCm9BRq7m/gwqrK8GcwOXDBPKWq21jFPg==;EndpointSuffix=core.windows.net'

In [5]:
connection_string = getpass.getpass()
# copy the above string without any quotes

········


In [6]:
container_client = azure.storage.blob.ContainerClient.from_connection_string(
    connection_string, container_name="naames"
)
# this will return True if the authentication works.
# if not, then it will show as [*] running for a while without any output
container_client.exists()

True

## Data Source


In [7]:
%%time
container_name = 'naames'
account_name = 'ifcb'
url_prefix = f"https://{account_name}.blob.core.windows.net/{container_name}/"

# csv_url = url_prefix + 'image-file-directory.csv'
csv_url = url_prefix + 'NAAMES-predicted-labels-model-cnn-v1-b3.csv'
inventory_df = pd.read_csv(csv_url)

len(inventory_df['pred_label'].unique())

CPU times: total: 10.4 s
Wall time: 30.6 s


10

In [12]:
cloud_urls = [
    f"https://{account_name}.blob.core.windows.net/{container_name}/{inventory_df['image_path'][i].split('NAAMES/')[1]}" 
    for i in trange(inventory_df.shape[0])
]

  0%|          | 0/5514006 [00:00<?, ?it/s]

## Load Model

In [10]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json

# Load the model architecture from JSON file
json_file_path = '../ml-workflow/model_ckpt/model-cnn-v1-b3.json'
with open(json_file_path, 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

# Load the model weights from H5 file
h5_file_path = '../ml-workflow/model_ckpt/model-cnn-v1-b3.h5'
loaded_model.load_weights(h5_file_path)

# Compile the loaded model if needed
# loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Now the 'loaded_model' contains both architecture and weights

# Example: Make predictions using the loaded model
# predictions = loaded_model.predict(your_input_data)


## Parallelized Process


In [ ]:
# Specify the batch size and number of workers
batch_size = 320
num_workers = 24  # Adjust based on your system's capabilities

# Split URLs into batches
start_index = 0
url_batches = [cloud_urls[i:i + batch_size] for i in range(start_index, len(cloud_urls)//5, batch_size)]

# Create an empty list to store probabilities
all_probabilities = []

# Create a list to store the order of URLs
url_order = []

# Define a function for parallel preprocessing
def preprocess_batch(order_index, batch_urls):
    idxs = []
    data = []
    for i, c_url in enumerate(tqdm(batch_urls, leave=False)):
        data.append(du.preprocess_input(np.expand_dims(imageio.v2.imread(c_url), axis=-1)))
        idxs.append(order_index*batch_size + i + start_index)
    
    result = (idxs, data)
    return result

try:
    # Use concurrent.futures for parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_order_index = {
            executor.submit(preprocess_batch, i, batch_urls): i 
            for i, batch_urls in enumerate(url_batches)
        }
        for future in tqdm(concurrent.futures.as_completed(future_to_order_index), total=len(url_batches)):
            order_index = future_to_order_index[future]
            try:
                # Get batch of images
                url_idxs, img_data = future.result()
                url_order.extend(url_idxs)
                
                # predict images
                batch_probabilities = loaded_model.predict(np.array(img_data), verbose=0)
                all_probabilities.extend(batch_probabilities)
            except Exception as e:
                print(f"Error processing batch {order_index}: {e}")

except KeyboardInterrupt:
    print("Process interrupted by user.")

# Sort results based on the original order of URLs
sorted_indices = np.argsort(url_order)
ordered_all_probs = np.vstack(all_probabilities)[sorted_indices]
# Now, 'ordered_all_probs' contains the concatenated probabilities for all URLs in the original order


  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

...

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

In [ ]:
all_probs = np.vstack(all_probabilities)
all_probs_df = pd.DataFrame(all_probs)
all_probs_df.index = url_order

inventory_df.rename(columns = {"Unnamed: 0": "Index"}).merge(
    all_probs_df.reset_index(), left_on='Index', right_on='index', how='inner').to_parquet("inventory_df_with_probs.parquet.gzip", index=False, compression='gzip')

## Serial Process


In [ ]:
batch_size = 320
all_probabilities = []
for i in trange(0,len(cloud_urls), batch_size):
    images = np.array([
        du.preprocess_input(np.expand_dims(imageio.v2.imread(c_url), axis=-1)) for c_url in tqdm(cloud_urls[i:i+batch_size])
    ])
    probabilities = loaded_model.predict(images)
    all_probabilities.append(probabilities)
    